<a href="https://colab.research.google.com/github/SummerLife/EmbeddedSystem/blob/master/MachineLearning/project/08_code_nn_from_scratch/coding_nn_with_bp_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding a Neural Network with Backpropagation from scratch

- Initialize Network
- Forward Propagate
- Back Propagate Error
- Train Network
- Predict
- Seeds Dataset Case Study

## 1. Initialize Network

In [16]:
from random import seed
from random import random

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{"weight": [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{"weight": [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

seed(1)
network = initialize_network(2,1,2)
for layer in network:
    print(layer)

[{'weight': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weight': [0.2550690257394217, 0.49543508709194095]}, {'weight': [0.4494910647887381, 0.651592972722763]}]
